In [1]:
import requests
import json
import pprint

In [2]:
r = requests.get('http://141.5.108.59:3000/ims/case/v1p0/CFDocuments')

In [3]:
r.json()

{'CFDocuments': [{'uri': 'http://141.5.108.59:3000/uri/1e2537a8-3b92-11ea-800f-0242ac1a0003',
   'identifier': '1e2537a8-3b92-11ea-800f-0242ac1a0003',
   'lastChangeDateTime': '2020-01-20T14:35:34+00:00',
   'creator': 'Kultusministerkonferenz',
   'title': 'Bildungsstandards im Fach Deutsch für den Primarbereich',
   'language': 'de'},
  {'uri': 'http://141.5.108.59:3000/uri/05a9d6f8-3b96-11ea-bc94-0242ac1a0003',
   'identifier': '05a9d6f8-3b96-11ea-bc94-0242ac1a0003',
   'lastChangeDateTime': '2020-01-20T15:19:09+00:00',
   'creator': 'Ministerium für Bildung, Wissenschaft, Weiterbildung und Kultur, Rheinland-Pfalz',
   'title': 'Teilrahmenplan Deutsch',
   'language': 'de'}]}

In [5]:
for item in r.json()['CFDocuments']:
    print(item['title'])
    print(item['identifier'])
    url = 'http://141.5.108.59:3000/ims/case/v1p0/CFPackages/' + item['identifier']
    r_item = requests.get(url)
    for item in r_item.json()['CFItems']:
        pprint.pprint(item)

Bildungsstandards im Fach Deutsch für den Primarbereich
1e2537a8-3b92-11ea-800f-0242ac1a0003
{'CFItemType': 'Standard',
 'CFItemTypeURI': {'identifier': '2d59c7f6-3b93-11ea-8ea9-0242ac1a0003',
                   'title': 'Standard',
                   'uri': 'http://141.5.108.59:3000/uri/2d59c7f6-3b93-11ea-8ea9-0242ac1a0003'},
 'conceptKeywords': ['Sprechen und Zuhören'],
 'educationLevel': ['04'],
 'fullStatement': 'Die mündliche Sprache ist ein zentrales Mittel aller '
                  'schulischen und außerschulischen Kommunikation. Sprechen '
                  'ist immer auch soziales Handeln.\r\n'
                  '\r\n'
                  'Die Kinder entwickeln eine demokratische Gesprächskultur '
                  'und erweitern ihre mündliche Sprachhandlungskompetenz. Sie '
                  'führen Gespräche, erzählen, geben und verarbeiten '
                  'Informationen, gestalten ihr Sprechen bewusst und leisten '
                  'mündliche Beiträge zum Unterricht.\r\

In [11]:
r = requests.get('http://141.5.108.59:3000/ims/case/v1p0/CFPackages/1e2537a8-3b92-11ea-800f-0242ac1a0003')
# pprint.pprint(r.json()['CFAssociations'])
for item in r.json()['CFAssociations']:
    pprint.pprint(f"OriginNode: {item['originNodeURI']}")
    pprint.pprint(f"destinationNode: {item['destinationNodeURI']}")
    

("OriginNode: {'title': 'Sprechen und Zuhören', 'identifier': "
 "'2d553cfe-3b93-11ea-9ddf-0242ac1a0003', 'uri': "
 "'http://141.5.108.59:3000/uri/2d553cfe-3b93-11ea-9ddf-0242ac1a0003'}")
("destinationNode: {'title': 'Bildungsstandards im Fach Deutsch für den "
 "Primarbereich', 'identifier': '1e2537a8-3b92-11ea-800f-0242ac1a0003', 'uri': "
 "'http://141.5.108.59:3000/uri/1e2537a8-3b92-11ea-800f-0242ac1a0003'}")
("OriginNode: {'title': 'Gespräche führen', 'identifier': "
 "'6cba87aa-3b93-11ea-97b6-0242ac1a0003', 'uri': "
 "'http://141.5.108.59:3000/uri/6cba87aa-3b93-11ea-97b6-0242ac1a0003'}")
("destinationNode: {'title': 'Sprechen und Zuhören', 'identifier': "
 "'2d553cfe-3b93-11ea-9ddf-0242ac1a0003', 'uri': "
 "'http://141.5.108.59:3000/uri/2d553cfe-3b93-11ea-9ddf-0242ac1a0003'}")
("OriginNode: {'title': 'zu anderen sprechen', 'identifier': "
 "'038cc8e6-3b94-11ea-9db2-0242ac1a0003', 'uri': "
 "'http://141.5.108.59:3000/uri/038cc8e6-3b94-11ea-9db2-0242ac1a0003'}")
("destinationNode: {

# Build Tree with anytree

parent Node => Destination
"name" => OriginNode

# Tuple Ansatz

In [23]:
import json
import requests
import pprint

r = requests.get('http://141.5.108.59:3000/ims/case/v1p0/CFPackages/1e2537a8-3b92-11ea-800f-0242ac1a0003')

competence_framework = []
for item in r.json()['CFAssociations']:
    if item['associationType'] == 'isChildOf':
        pprint.pprint(f"This item has title {item['originNodeURI']['title']}")
        pprint.pprint(f"This item is child of {item['destinationNodeURI']['title']}")
        title_uri_dest = {'title': item['destinationNodeURI']['title'], 'uri': item['destinationNodeURI']['uri']}
        title_uri_orig = {'title': item['originNodeURI']['title'], 'uri': item['originNodeURI']['uri']}
        competence_framework.append(
            (item['destinationNodeURI']['title'],(item['originNodeURI']['title']))
            )

print(competence_framework)

parents, children = zip(*competence_framework)
root_nodes = {x for x in parents if x not in children}
print(root_nodes)
for node in root_nodes:
    competence_framework.append(('Root', node))

def get_nodes(node):
    d = {}
    d['name'] = node
    children = get_children(node)
    if children:
        d['children'] = [get_nodes(child) for child in children]
    return d

def get_children(node):
    return [x[1] for x in competence_framework if x[0] == node]

tree = get_nodes('Root')
print(json.dumps(tree, indent=4))

'This item has title Sprechen und Zuhören'
'This item is child of Bildungsstandards im Fach Deutsch für den Primarbereich'
'This item has title Gespräche führen'
'This item is child of Sprechen und Zuhören'
'This item has title zu anderen sprechen'
'This item is child of Sprechen und Zuhören'
'This item has title verstehend zuhören'
'This item is child of Sprechen und Zuhören'
'This item has title szenisch spielen'
'This item is child of Sprechen und Zuhören'
'This item has title über Lernen sprechen'
'This item is child of Sprechen und Zuhören'
'This item has title Test'
'This item is child of Bildungsstandards im Fach Deutsch für den Primarbereich'
'This item has title Test-Child'
'This item is child of Test'
'This item has title Test-Child-Child'
'This item is child of Test-Child'
[('Bildungsstandards im Fach Deutsch für den Primarbereich', 'Sprechen und Zuhören'), ('Sprechen und Zuhören', 'Gespräche führen'), ('Sprechen und Zuhören', 'zu anderen sprechen'), ('Sprechen und Zuhören',

In [7]:
import json

parents, children = zip(*competence_framework)
root_nodes = {x for x in parents if x not in children}
print(root_nodes)
for node in root_nodes:
    competence_framework.append(('Root', node))

def get_nodes(node):
    d = {}
    d['name'] = node
    children = get_children(node)
    if children:
        d['children'] = [get_nodes(child) for child in children]
    return d

def get_children(node):
    return [x[1] for x in competence_framework if x[0] == node]

tree = get_nodes('Root')
print(json.dumps(tree, indent=4))

{'Root'}


RecursionError: maximum recursion depth exceeded in comparison

In [51]:
competence_list = []
competencies = sorted(competence_framework, key=itemgetter('isChildOf'))

for key, value in itertools.groupby(competencies, key=lambda x:x['isChildOf']):
    print(key)
    print(list(value))
    competence_list.append({'name': key, 'value': list(value)})
print(competence_list)

Sprechen und Zuhören
[{'name': 'zu anderen sprechen', 'isChildOf': 'Sprechen und Zuhören'}, {'name': 'Gespräche führen', 'isChildOf': 'Sprechen und Zuhören'}, {'name': 'verstehend zuhören', 'isChildOf': 'Sprechen und Zuhören'}, {'name': 'mit verteilten Rollen sprechen/spielen', 'isChildOf': 'Sprechen und Zuhören'}, {'name': 'über Lernerfahrungen sprechen', 'isChildOf': 'Sprechen und Zuhören'}]
Teilrahmenplan Deutsch
[{'name': 'Sprechen und Zuhören', 'isChildOf': 'Teilrahmenplan Deutsch'}]
[{'name': 'Sprechen und Zuhören', 'value': []}, {'name': 'Teilrahmenplan Deutsch', 'value': []}]
